In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

train_dir="chimuf/train"
val_dir="chimuf/validate"
test_dir="chimuf/test"
classes=['chihuahua','muffin']

#below lines of code are for creating validation dataset 
# its ok to not run it if you are using my provided dataset 
'''
for cls in classes:
  os.makedirs(os.path.join(val_dir,cls),exist_ok=True)

for cls in classes:
  cls_path=os.path.join(train_dir,cls)
  images=os.listdir(cls_path)
  train_img,val_img=train_test_split(images,test_size=0.2,random_state=42)

  for f in val_img:
    shutil.move(os.path.join(cls_path,f),os.path.join(val_dir,cls,f))
'''


'\nfor cls in classes:\n  os.makedirs(os.path.join(val_dir,cls),exist_ok=True)\n\nfor cls in classes:\n  cls_path=os.path.join(train_dir,cls)\n  images=os.listdir(cls_path)\n  train_img,val_img=train_test_split(images,test_size=0.2,random_state=42)\n\n  for f in val_img:\n    shutil.move(os.path.join(cls_path,f),os.path.join(val_dir,cls,f))\n'

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen=ImageDataGenerator(
    rescale=1./255
)
test_datagen=ImageDataGenerator(
    rescale=1./255
)

train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)
val_generator=val_datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)
test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)
print("Classes found:", train_generator.class_indices)


c:\Python Setup\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Python Setup\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Python Setup\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  

Found 1237 images belonging to 2 classes.
Found 311 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Classes found: {'chihuahua': 0, 'muffin': 1}


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model=MobileNetV2(weights='imagenet',include_top=False,input_shape=(224,224,3))

base_model.trainable=False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)  # helps prevent overfitting
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output_layer = Dense(1, activation='sigmoid')(x)  # we used sigmoid for binary output (0 or 1)

model = Model(inputs=base_model.input, outputs=output_layer)

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    verbose=1
)
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")



9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


c:\Python Setup\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.8076 - loss: 0.4167 - val_accuracy: 0.9807 - val_loss: 0.1499
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 30s 771ms/step - accuracy: 0.9580 - loss: 0.1476 - val_accuracy: 0.9807 - val_loss: 0.0876
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 29s 756ms/step - accuracy: 0.9733 - loss: 0.0975 - val_accuracy: 0.9807 - val_loss: 0.0726
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 32s 816ms/step - accuracy: 0.9749 - loss: 0.0779 - val_accuracy: 0.9807 - val_loss: 0.0666
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 30s 776ms/step - accuracy: 0.9814 - loss: 0.0591 - val_accuracy: 0.9807 - val_loss: 0.0632
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 30s 761ms/step - accuracy: 0.9798 - loss: 0.0609 - val_accuracy: 0.9839 - val_loss: 0.0605
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 30s 760ms/step - accuracy: 0.9798 - loss: 0.0532 - val_accuracy: 0.9839 - val_loss: 0.0596
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 30s 760ms/step - accuracy: 0.9879 - loss: 0.0445 - val_accurac

In [8]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "chimuf/test/muffin/img_0_423.jpg"  # path to your image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0  # normalize

prediction = model.predict(img_array)[0][0]

if prediction > 0.5:
    print("Predicted: Muffin")
else:
    print("Predicted: Chihuahua")

print("Confidence:", round(float(prediction), 3))

model.save("bin_classifier.h5")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


Predicted: Muffin
Confidence: 0.999
